In [ ]:
# Using time and day as numeric data without dummy va

In [20]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression, LogisticRegression
import pickle
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm


In [2]:
SQLPW=''

In [3]:
# Connect and query
engine = create_engine("mysql+mysqlconnector://softies:" + SQLPW + "@db-bikes.ck7tnbvjxsza.eu-west-1.rds.amazonaws.com:3306/db-bikes")

# Test with just station 108
stations = pd.read_sql_query("SELECT dynamic.available_bikes, dynamic.last_update from dynamic JOIN static ON static.address=dynamic.address WHERE static.number=108", engine)

In [4]:
stations

,available_bikes,last_update
0,9,2022-02-22 15:43:46
1,9,2022-02-22 15:53:52
2,18,2022-02-23 18:03:46
3,17,2022-02-23 18:17:57
4,17,2022-02-23 18:28:04
...,...,...
8244,16,2022-04-11 19:32:53
8245,17,2022-04-11 19:42:46
8246,17,2022-04-11 19:52:53
8247,16,2022-04-11 19:54:56


In [11]:
features = ['time', 'day']
days_of_week = ["Sunday","Monday", "Tuesday","Wednesday", "Thursday", "Friday", "Saturday"]

# Split last update to day and time
stations['day'] = list(map(lambda x: x.strftime('%A'), list(stations['last_update'])))
stations['time'] = list(map(lambda x: x.strftime('%H'), list(stations['last_update'])))

# Another sexual lambda function to convert day of week to number
stations['day'] = list(map(lambda x: days_of_week.index(x), list(stations['day'])))

stations

,available_bikes,last_update,day,time
0,9,2022-02-22 15:43:46,2,15
1,9,2022-02-22 15:53:52,2,15
2,18,2022-02-23 18:03:46,3,18
3,17,2022-02-23 18:17:57,3,18
4,17,2022-02-23 18:28:04,3,18
...,...,...,...,...
8244,16,2022-04-11 19:32:53,1,19
8245,17,2022-04-11 19:42:46,1,19
8246,17,2022-04-11 19:52:53,1,19
8247,16,2022-04-11 19:54:56,1,19


In [12]:
X = stations[features]
X

,time,day
0,15,2
1,15,2
2,18,3
3,18,3
4,18,3
...,...,...
8244,19,1
8245,19,1
8246,19,1
8247,19,1


In [14]:
Y = stations['available_bikes']

In [16]:
# time and day are both left as integers with no dummy values
X

,time,day
0,15,2
1,15,2
2,18,3
3,18,3
4,18,3
...,...,...
8244,19,1
8245,19,1
8246,19,1
8247,19,1


In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 40)


In [19]:
model = LinearRegression()
model.fit(X_train, Y_train)
y_test_pred = model.predict(X_test)

In [21]:
print("Mean absolute error =", round(sm.mean_absolute_error(Y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(Y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(Y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(Y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(Y_test, y_test_pred), 2))

Mean absolute error = 6.49
Mean squared error = 60.24
Median absolute error = 5.87
Explain variance score = 0.01
R2 score = 0.01
